In [32]:
filename = "requirements.txt"
file_content = """gymnasium>=0.29.0
stable-baselines3>=2.0.0
sb3-contrib>=2.0.0
numpy>=1.24.0
pandas>=2.0.0
tqdm>=4.65.0
tensorboard>=2.14.0
 """

with open(filename, "w") as f:
    f.write(file_content)

print(f"파일 '{filename}'이(가) 성공적으로 저장되었습니다.")

파일 'requirements.txt'이(가) 성공적으로 저장되었습니다.


In [33]:
!pip install -r requirements.txt
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!rm requirements.txt

Looking in indexes: https://download.pytorch.org/whl/cu124


In [34]:
import torch

if torch.cuda.is_available():
    print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is NOT available to PyTorch.")
    print("This could be due to a missing or misconfigured CUDA toolkit, or driver issues.")

CUDA is available! Using GPU: NVIDIA L4


In [35]:
# Source: /content/config.py
"""Experiment configuration for superpermutation RL research."""

from typing import Dict, List, Tuple

# Permutation sizes to test
N_LIST: List[int] = [5] #3,4,

# Environment types
ENV_TYPES: List[str] = [ "word_cost","symbol"]

# Algorithms to test
ALGOS: List[str] = ["ppo"   ] #"maskable_ppo","dqn_dueling", "a2c","dqn",

# Random seeds for experiments
SEEDS: List[int] = [0] #0, 1, 2,, , 123, 999 42

# Hyperparameter grids (for learning algorithms only)
HYPER_GRIDS: Dict[str, Dict[str, List[float]]] = {
    "ppo": {
        "learning_rate": [2e-4],
        "gamma": [0.99],
    },
    "dqn": {
        "learning_rate": [1e-4],
        "gamma": [0.99],
    },
}

# Total timesteps per algorithm
TIMESTEPS: Dict[str, int] = {
    "ppo": 2_200_000,
    "dqn": 2_200_000,
}

# Episodes for non-learning baselines
BASELINE_EPISODES: Dict[str, int] = {
    "random": 300,
    "greedy": 300,
}

# Conditions for skipping certain algorithm/environment combinations
# Format: (algo_name, env_type, n) -> skip
SKIP_CONDITIONS: List[Tuple[str, str, int]] = [
    # Skip DQN variants for word_cost and n=6 (too heavy)
    ("dqn", "word_cost", 6),

]


def should_skip(algo_name: str, env_type: str, n: int) -> bool:
    """
    Check if a combination should be skipped.

    Args:
        algo_name: Algorithm name
        env_type: Environment type
        n: Permutation size

    Returns:
        True if should skip, False otherwise
    """
    return (algo_name, env_type, n) in SKIP_CONDITIONS



In [36]:
# Source: /content/callbacks.py
"""Stable-Baselines3 callbacks for superpermutation training."""

import os
import json
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback
from typing import Dict, Any, Optional




class SuperpermEpisodeCallback(BaseCallback):
    """
    Callback to log episode-level metrics for superpermutation environments.

    Logs:
    - superperm/final_length
    - superperm/success (0/1)
    - superperm/coverage_ratio
    - superperm/episode_steps
    - superperm/new_perms_total (if available)
    - superperm/duplicate_action_ratio
    - superperm/episode_return
    - superperm/best_final_length_so_far
    """

    def __init__(self, verbose: int = 0, save_episodes_dir: Optional[str] = None, top_k: int = 5):
        super().__init__(verbose)
        self.best_final_length_so_far = float("inf")
        self.episode_count = 0
        self.episode_actions = []  # Track actions for duplicate ratio
        self.episode_return = 0.0
        self.episode_steps = 0
        self.save_episodes_dir = save_episodes_dir
        self.top_k = top_k  # Number of top episodes to save
        self.top_episodes = []  # List of (final_length, episode_data) tuples, sorted by length
        if self.save_episodes_dir:
            os.makedirs(self.save_episodes_dir, exist_ok=True)

    def _on_step(self) -> bool:
        """Called at each step."""
        # Track actions for duplicate ratio
        if self.locals.get("actions") is not None:
            action = self.locals["actions"][0] if isinstance(self.locals["actions"], (list, np.ndarray)) else self.locals["actions"]
            self.episode_actions.append(int(action))

        # Accumulate reward
        if "rewards" in self.locals:
            reward = self.locals["rewards"][0] if isinstance(self.locals["rewards"], (list, np.ndarray)) else self.locals["rewards"]
            self.episode_return += float(reward)

        self.episode_steps += 1

        # Check if episode is done
        dones = self.locals.get("dones", [False])
        terminated = self.locals.get("terminated", [False])
        truncated = self.locals.get("truncated", [False])

        # Check if any episode ended
        if isinstance(dones, (list, np.ndarray)):
            done = bool(dones[0]) if len(dones) > 0 else False
        else:
            done = bool(dones)

        if isinstance(terminated, (list, np.ndarray)):
            term = bool(terminated[0]) if len(terminated) > 0 else False
        else:
            term = bool(terminated)

        if isinstance(truncated, (list, np.ndarray)):
            trunc = bool(truncated[0]) if len(truncated) > 0 else False
        else:
            trunc = bool(truncated)

        if done or term or trunc:
            self._on_episode_end()

        return True

    def _on_episode_end(self) -> None:
        """Called when an episode ends."""
        # Get info from the environment
        infos = self.locals.get("infos", [{}])
        if isinstance(infos, (list, np.ndarray)) and len(infos) > 0:
            info = infos[0]
        else:
            info = {}

        # Extract metrics from info
        final_length = info.get("final_length", self.episode_steps)
        success = 1 if info.get("success", False) else 0
        coverage_ratio = info.get("coverage_ratio", 0.0)
        sequence = info.get("sequence", [])

        # Compute duplicate action ratio
        if len(self.episode_actions) > 1:
            unique_actions = len(set(self.episode_actions))
            duplicate_ratio = 1.0 - (unique_actions / len(self.episode_actions))
        else:
            duplicate_ratio = 0.0

        # Update best final length
        if success and final_length < self.best_final_length_so_far:
            self.best_final_length_so_far = final_length

        # Log metrics
        self.logger.record("superperm/final_length", float(final_length))
        self.logger.record("superperm/success", float(success))
        self.logger.record("superperm/coverage_ratio", float(coverage_ratio))
        self.logger.record("superperm/episode_steps", float(self.episode_steps))
        self.logger.record("superperm/episode_return", float(self.episode_return))
        self.logger.record("superperm/duplicate_action_ratio", float(duplicate_ratio))
        self.logger.record("superperm/best_final_length_so_far", float(self.best_final_length_so_far))

        # New perms total (if available in info)
        if "new_perms_total" in info:
            self.logger.record("superperm/new_perms_total", float(info["new_perms_total"]))

        # Save episode info if directory is provided (for successful episodes)
        # Only keep top-k shortest episodes
        if self.save_episodes_dir and success and sequence:
            episode_data = {
                "episode": self.episode_count,
                "final_length": int(final_length),
                "episode_return": float(self.episode_return),
                "sequence": list(sequence) if isinstance(sequence, (list, tuple)) else sequence,
                "success": bool(success),
                "coverage_ratio": float(coverage_ratio),
            }

            # Add to top episodes list
            self.top_episodes.append((final_length, episode_data))

            # Sort by final_length (ascending), then by episode_return (descending)
            self.top_episodes.sort(key=lambda x: (x[0], -x[1]["episode_return"]))

            # Keep only top-k
            if len(self.top_episodes) > self.top_k:
                self.top_episodes = self.top_episodes[:self.top_k]

            # Save all top-k episodes
            self._save_top_episodes()

        # Reset episode tracking
        self.episode_count += 1
        self.episode_actions = []
        self.episode_return = 0.0
        self.episode_steps = 0

    def _save_top_episodes(self) -> None:
        """Save the top-k episodes to disk."""
        if not self.save_episodes_dir:
            return

        # Remove old episode files
        try:
            for file in os.listdir(self.save_episodes_dir):
                if file.startswith("episode_") and file.endswith(".json"):
                    os.remove(os.path.join(self.save_episodes_dir, file))
        except (OSError, FileNotFoundError):
            pass  # Directory might not exist or be empty

        # Save current top-k episodes
        for rank, (final_length, episode_data) in enumerate(self.top_episodes, 1):
            episode_data["rank"] = rank
            # Convert numpy types to Python native types for JSON serialization
            episode_data_serializable = convert_numpy_types(episode_data)
            episode_path = os.path.join(self.save_episodes_dir, f"episode_{episode_data['episode']}.json")
            with open(episode_path, "w") as f:
                json.dump(episode_data_serializable, f, indent=2)



In [37]:
# Source: /content/algorithms/greedy_policy.py
"""Greedy policy baseline."""

import os
import csv
import numpy as np
import gymnasium as gym
from tqdm import tqdm




def run_greedy(
    env: gym.Env,
    total_episodes: int,
    max_steps_per_episode: int,
    log_dir: str,
    seed: int,
) -> None:
    """
    Run greedy policy baseline.

    For SymbolEnv: selects action that maximizes expected new permutations.
    For WordEnv: selects action with minimum cost among uncovered permutations.

    Args:
        env: Gymnasium environment
        total_episodes: Number of episodes to run
        max_steps_per_episode: Maximum steps per episode
        log_dir: Directory to save logs
        seed: Random seed
    """
    os.makedirs(log_dir, exist_ok=True)

    # Set seed
    np.random.seed(seed)

    # CSV file for episode metrics
    csv_path = os.path.join(log_dir, "progress.csv")

    # Field names matching SB3 format
    fieldnames = [
        "episode",
        "final_length",
        "success",
        "coverage_ratio",
        "episode_steps",
        "episode_return",
        "covered_permutations",
    ]

    # Detect environment type
    detected_env_type = detect_env_type(env)
    # Map to internal representation for greedy action selection
    env_type = "word" if detected_env_type == "word_cost" else "symbol"

    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for episode in tqdm(range(total_episodes), desc="Greedy policy episodes"):
            # Reset with seed offset
            obs, info = env.reset(seed=seed + episode)

            episode_return = 0.0
            episode_steps = 0
            done = False

            while not done and episode_steps < max_steps_per_episode:
                if env_type == "symbol":
                    action = _greedy_action_symbol(env, obs)
                elif env_type == "word":
                    action = _greedy_action_word(env, obs)
                else:
                    # Fallback to random
                    action = env.action_space.sample()

                obs, reward, terminated, truncated, info = env.step(action)

                episode_return += reward
                episode_steps += 1
                done = terminated or truncated

            # Write episode metrics
            writer.writerow({
                "episode": episode,
                "final_length": info.get("final_length", len(env.string) if hasattr(env, "string") else 0),
                "success": 1 if info.get("success", False) else 0,
                "coverage_ratio": info.get("coverage_ratio", 0.0),
                "episode_steps": episode_steps,
                "episode_return": episode_return,
                "covered_permutations": info.get("covered_permutations", 0),
            })
            f.flush()

    # Save metadata
    n = env.n if hasattr(env, "n") else 0
    metadata = create_metadata(
        env_type=detected_env_type,
        n=n,
        algo_name="greedy",
        hyperparams={},
        seed=seed,
        total_episodes=total_episodes,
    )
    save_metadata(log_dir, metadata)


def _greedy_action_symbol(env: gym.Env, obs: dict) -> int:
    """
    Greedy action selection for symbol environment.

    For each possible action, estimate the number of new permutations
    that would be discovered, and select the action with the highest score.
    """
    best_action = 0
    best_score = float("-inf")

    # Create a temporary copy of coverage for simulation
    temp_coverage = env.coverage.copy()
    temp_string = env.string.copy()

    for action in range(env.action_space.n):
        # Simulate adding this symbol
        symbol = action + 1
        test_string = temp_string + [symbol]

        # Estimate new permutations (simplified: check if this creates new length-n substrings)
        # We'll use a heuristic: count how many new permutations might be formed
        # by checking the last n symbols
        new_perms_estimate = 0
        if len(test_string) >= env.n:
            # Check the last n symbols
            last_n = tuple(test_string[-env.n:])
            if last_n in env.perms:
                idx = env.perms.index(last_n)
                if not temp_coverage[idx]:
                    new_perms_estimate = 1

        # Score: new_perms - small length penalty
        score = new_perms_estimate - 0.01  # Small penalty for length

        if score > best_score:
            best_score = score
            best_action = action

    return best_action


def _greedy_action_word(env: gym.Env, obs: dict) -> int:
    """
    Greedy action selection for word environment.

    Select the action (permutation) with minimum cost among uncovered ones.
    If all are covered, select the one with minimum cost.
    """
    coverage = obs["coverage"]
    costs = obs["costs"]

    # Prefer uncovered permutations
    uncovered_mask = coverage == 0
    uncovered_costs = costs.copy()
    uncovered_costs[~uncovered_mask] = float("inf")  # Mask covered ones

    if np.any(uncovered_mask):
        # Select uncovered permutation with minimum cost
        best_action = int(np.argmin(uncovered_costs))
    else:
        # All covered, select minimum cost overall
        best_action = int(np.argmin(costs))

    return best_action




In [38]:
# Source: /content/algorithms/random_policy.py
"""Random policy baseline."""

import os
import csv
import numpy as np
import gymnasium as gym
from tqdm import tqdm




def run_random(
    env: gym.Env,
    total_episodes: int,
    max_steps_per_episode: int,
    log_dir: str,
    seed: int,
) -> None:
    """
    Run random policy baseline.

    Args:
        env: Gymnasium environment
        total_episodes: Number of episodes to run
        max_steps_per_episode: Maximum steps per episode
        log_dir: Directory to save logs
        seed: Random seed
    """
    os.makedirs(log_dir, exist_ok=True)

    # Set seed
    np.random.seed(seed)

    # CSV file for episode metrics
    csv_path = os.path.join(log_dir, "progress.csv")

    # Field names matching SB3 format
    fieldnames = [
        "episode",
        "final_length",
        "success",
        "coverage_ratio",
        "episode_steps",
        "episode_return",
        "covered_permutations",
    ]

    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for episode in tqdm(range(total_episodes), desc="Random policy episodes"):
            # Reset with seed offset
            obs, info = env.reset(seed=seed + episode)

            episode_return = 0.0
            episode_steps = 0
            done = False

            while not done and episode_steps < max_steps_per_episode:
                # Sample random action
                action = env.action_space.sample()
                obs, reward, terminated, truncated, info = env.step(action)

                episode_return += reward
                episode_steps += 1
                done = terminated or truncated

            # Write episode metrics
            writer.writerow({
                "episode": episode,
                "final_length": info.get("final_length", len(env.string) if hasattr(env, "string") else 0),
                "success": 1 if info.get("success", False) else 0,
                "coverage_ratio": info.get("coverage_ratio", 0.0),
                "episode_steps": episode_steps,
                "episode_return": episode_return,
                "covered_permutations": info.get("covered_permutations", 0),
            })
            f.flush()

    # Save metadata
    env_type = detect_env_type(env)
    n = env.n if hasattr(env, "n") else 0
    metadata = create_metadata(
        env_type=env_type,
        n=n,
        algo_name="random",
        hyperparams={},
        seed=seed,
        total_episodes=total_episodes,
    )
    save_metadata(log_dir, metadata)




In [39]:
# Source: /content/envs/symbol_env.py
"""Symbol-based superpermutation environment."""

import math
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from typing import Dict, Any, Optional



class SymbolSuperpermEnv(gym.Env):
    """
    Symbol-based superpermutation environment.

    At each step, the agent adds one symbol (1 to n) to the sequence.
    The goal is to construct a sequence that contains all n! permutations
    as contiguous substrings while minimizing the total length.
    """

    metadata = {"render_modes": []}

    def __init__(
        self,
        n: int = 4,
        max_length: int | None = None,
        max_steps: int | None = None,
        lambda_length_penalty: float = 0.01,
        alpha_new_perm_reward: float = 5.0,
        goal_bonus: float = 200.0,
        length_penalty_at_goal: float = 0.0,
    ):
        """
        Initialize the symbol-based superpermutation environment.

        Args:
            n: Alphabet size (permutations of {1, 2, ..., n})
            max_length: Maximum allowed sequence length (default: (n/2) * n!)
            max_steps: Maximum number of steps (default: max_length)
            lambda_length_penalty: Penalty per step
            alpha_new_perm_reward: Reward per newly discovered permutation
            goal_bonus: Bonus reward when all permutations are covered
            length_penalty_at_goal: Additional penalty based on final length at goal
        """
        super().__init__()

        self.n = n
        self.alphabet = list(range(1, n + 1))
        self.perms = get_all_permutations(n)
        self.m = len(self.perms)  # n!

        # Set defaults
        if max_length is None:
            max_length = int((n/2) * math.factorial(n))
        if max_steps is None:
            max_steps = 2*max_length

        self.max_length = max_length
        self.max_steps = max_steps
        self.lambda_length_penalty = lambda_length_penalty
        self.alpha_new_perm_reward = alpha_new_perm_reward
        self.goal_bonus = goal_bonus
        self.length_penalty_at_goal = length_penalty_at_goal

        # Internal state
        self.string: list[int] = []
        self.coverage: np.ndarray = np.zeros(self.m, dtype=bool)
        self.step_count: int = 0

        # Action space: choose one symbol from {1, ..., n}
        self.action_space = spaces.Discrete(n)

        # Observation space
        self.observation_space = spaces.Dict({
            "suffix": spaces.MultiDiscrete([n + 1] * (n - 1)),  # 0-padded, values in {0,...,n}
            "coverage": spaces.MultiBinary(self.m),
            "length": spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32),
        })

    def _get_observation(self) -> Dict[str, Any]:
        """Build the current observation."""
        # Suffix: last (n-1) symbols, 0-padded
        suffix = np.zeros(self.n - 1, dtype=np.int32)
        if len(self.string) > 0:
            suffix_start = max(0, len(self.string) - (self.n - 1))
            suffix_len = len(self.string) - suffix_start
            suffix[-suffix_len:] = self.string[suffix_start:]

        # Coverage: boolean array
        coverage = self.coverage.copy().astype(np.int8)

        # Length: normalized to [0, 1]
        length_norm = np.array([min(1.0, len(self.string) / self.max_length)], dtype=np.float32)

        return {
            "suffix": suffix,
            "coverage": coverage,
            "length": length_norm,
        }

    def reset(
        self,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
    ) -> tuple[Dict[str, Any], Dict[str, Any]]:
        """Reset the environment."""
        super().reset(seed=seed)

        self.string = []
        self.coverage[:] = False
        self.step_count = 0

        obs = self._get_observation()
        info = {"n": self.n}

        return obs, info

    def step(self, action: int) -> tuple[Dict[str, Any], float, bool, bool, Dict[str, Any]]:
        """
        Execute one step.

        Returns:
            observation, reward, terminated, truncated, info
        """
        # Validate action
        if not self.action_space.contains(action):
            raise ValueError(f"Invalid action: {action}")

        # Map action to symbol: action ∈ {0,...,n-1} -> symbol ∈ {1,...,n}
        symbol = action + 1

        # Append symbol to string
        self.string.append(symbol)
        self.step_count += 1

        # Update coverage
        delta_new_perms = update_coverage_for_string(
            string=self.string,
            n=self.n,
            perms=self.perms,
            coverage=self.coverage,
            search_window=2 * self.n,  # Only check recent suffix
        )

        # Compute reward
        new_perms_reward = self.alpha_new_perm_reward * delta_new_perms
        step_length_penalty = -self.lambda_length_penalty
        reward = new_perms_reward + step_length_penalty

        # Check termination conditions
        success = False
        truncated = False
        terminated = False

        if self.coverage.sum() == self.m:
            # All permutations covered
            success = True
            terminated = True
            final_length = len(self.string)
            reward += self.goal_bonus - self.length_penalty_at_goal * final_length
        elif len(self.string) > self.max_length or self.step_count >= self.max_steps:
            truncated = True
            terminated = True
            reward-=100
        else:
            terminated = False

        # Build observation
        obs = self._get_observation()

        # Build info
        info = {
            "n": self.n,
            "step_count": self.step_count,
            "current_length": len(self.string),
            "coverage_ratio": float(self.coverage.sum()) / float(self.m),
            "success": success,
            "truncated": truncated,
        }

        if terminated or truncated:
            info["final_length"] = len(self.string)
            info["sequence"] = list(self.string)
            info["covered_permutations"] = int(self.coverage.sum())

        return obs, reward, terminated, truncated, info




In [40]:
# Source: /content/envs/utils.py
"""Utility functions for superpermutation environments."""

import numpy as np
from typing import List, Tuple
from itertools import permutations


def get_all_permutations(n: int) -> List[Tuple[int, ...]]:
    """
    Generate all permutations of the alphabet {1, 2, ..., n}.

    Args:
        n: Size of the alphabet

    Returns:
        List of all permutations as tuples
    """
    alphabet = list(range(1, n + 1))
    return list(permutations(alphabet))


def max_overlap(seq: List[int], word: List[int]) -> int:
    """
    Compute the maximum overlap length between seq and word.

    The overlap is the length ℓ such that the last ℓ symbols of seq
    equal the first ℓ symbols of word.

    Args:
        seq: Current sequence
        word: Word to check overlap with

    Returns:
        Maximum overlap length (0 to min(len(seq), len(word)))
    """
    if len(seq) == 0 or len(word) == 0:
        return 0

    max_len = min(len(seq), len(word))
    for overlap_len in range(max_len, 0, -1):
        if seq[-overlap_len:] == word[:overlap_len]:
            return overlap_len
    return 0


def merge_with_overlap(seq: List[int], word: List[int]) -> List[int]:
    """
    Merge word into seq using maximum overlap.

    Args:
        seq: Current sequence
        word: Word to merge

    Returns:
        Merged sequence: seq + word[overlap:]
    """
    overlap = max_overlap(seq, word)
    return seq + word[overlap:]


def update_coverage_for_string(
    string: List[int],
    n: int,
    perms: List[Tuple[int, ...]],
    coverage: np.ndarray,
    search_window: int | None = None,
) -> int:
    """
    Update coverage array by scanning string for contiguous substrings of length n.

    Args:
        string: The sequence to scan
        n: Length of permutations
        perms: List of all permutations (tuples)
        coverage: Boolean array of shape (len(perms),) to update
        search_window: If not None, only scan the last search_window symbols

    Returns:
        Number of newly discovered permutations (positions that changed from False to True)
    """
    if len(string) < n:
        return 0

    # Determine search range
    if search_window is not None:
        start_idx = max(0, len(string) - search_window)
    else:
        start_idx = 0

    delta_new = 0
    # Scan all length-n substrings in the search window
    for i in range(start_idx, len(string) - n + 1):
        substring = tuple(string[i:i+n])
        # Find index in perms
        try:
            idx = perms.index(substring)
            if not coverage[idx]:
                coverage[idx] = True
                delta_new += 1
        except ValueError:
            # Substring is not a valid permutation, skip
            pass

    return delta_new


def flatten_symbol_observation(obs: dict, n: int, m: int) -> np.ndarray:
    """
    Flatten symbol environment observation for neural networks.

    Args:
        obs: Observation dict with keys "suffix", "coverage", "length"
        n: Alphabet size
        m: Number of permutations

    Returns:
        1D numpy array
    """
    suffix = obs["suffix"].astype(np.float32)
    coverage = obs["coverage"].astype(np.float32)
    length = obs["length"].astype(np.float32)
    return np.concatenate([suffix, coverage, length])


def flatten_word_observation(obs: dict, n: int, m: int) -> np.ndarray:
    """
    Flatten word environment observation for neural networks.

    Args:
        obs: Observation dict with keys "coverage", "costs", "length"
        n: Alphabet size
        m: Number of permutations

    Returns:
        1D numpy array
    """
    coverage = obs["coverage"].astype(np.float32)
    costs = obs["costs"].astype(np.float32)
    length = obs["length"].astype(np.float32)
    return np.concatenate([coverage, costs, length])


def canonicalize_superperm(sequence: List[int], n: int) -> List[int]:
    """
    Canonicalize a superpermutation by relabeling to a standard form.

    Approach:
    1. Find the first contiguous block of length n that is a permutation of {1,...,n}
    2. Use that block to define a relabeling mapping
    3. Apply the mapping to the entire sequence

    Args:
        sequence: The superpermutation sequence
        n: Alphabet size

    Returns:
        Canonicalized sequence
    """
    if len(sequence) < n:
        return sequence.copy()

    # Find first permutation block
    mapping = None
    for i in range(len(sequence) - n + 1):
        block = sequence[i:i+n]
        # Check if block is a permutation of {1,...,n}
        if set(block) == set(range(1, n + 1)) and len(block) == len(set(block)):
            # Create mapping: block[j] -> j+1
            mapping = {block[j]: j + 1 for j in range(n)}
            break

    # If no permutation block found, return original
    if mapping is None:
        return sequence.copy()

    # Apply mapping
    canonical = [mapping.get(symbol, symbol) for symbol in sequence]
    return canonical




In [41]:
# Source: /content/envs/word_env_cost.py
"""Word-based superpermutation environment with cost state."""

import math
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from typing import Dict, Any, Optional



class WordCostSuperpermEnv(gym.Env):
    """
    Word-based superpermutation environment with cost state.

    At each step, the agent selects one permutation to add to the sequence.
    The cost of adding a permutation is (n - overlap), where overlap is
    the maximum overlap between the current sequence and the selected permutation.
    """

    metadata = {"render_modes": []}

    def __init__(
        self,
        n: int = 4,
        max_length: int | None = None,
        max_steps: int | None = None,
        alpha_new_perm_reward: float = 5.0,
        goal_bonus: float = 200.0,
        length_penalty_at_goal: float = 0.0,
    ):
        """
        Initialize the word-based superpermutation environment.

        Args:
            n: Alphabet size (permutations of {1, 2, ..., n})
            max_length: Maximum allowed sequence length (default: (n/2) * n!)
            max_steps: Maximum number of steps (default: 3 * n!)
            alpha_new_perm_reward: Reward per newly discovered permutation
            goal_bonus: Bonus reward when all permutations are covered
            length_penalty_at_goal: Additional penalty based on final length at goal
        """
        super().__init__()

        self.n = n
        self.perms = get_all_permutations(n)
        self.m = len(self.perms)  # n!
        self.perms_as_lists = [list(p) for p in self.perms]

        # Set defaults
        if max_length is None:
            max_length = int((n/2) * math.factorial(n))
        if max_steps is None:
            max_steps = 3*max_length

        self.max_length = max_length
        self.max_steps = max_steps
        self.alpha_new_perm_reward = alpha_new_perm_reward
        self.goal_bonus = goal_bonus
        self.length_penalty_at_goal = length_penalty_at_goal

        # Internal state
        self.string: list[int] = []
        self.coverage: np.ndarray = np.zeros(self.m, dtype=bool)
        self.step_count: int = 0
        self.costs: np.ndarray = np.zeros(self.m, dtype=np.float32)

        # Action space: choose one permutation from {0, ..., m-1}
        self.action_space = spaces.Discrete(self.m)

        # Observation space
        self.observation_space = spaces.Dict({
            "coverage": spaces.MultiBinary(self.m),
            "costs": spaces.Box(low=0.0, high=float(self.n), shape=(self.m,), dtype=np.float32),
            "length": spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32),
        })

    def _compute_cost_vector(self) -> np.ndarray:
        """
        Compute the cost vector for all permutations.

        Cost[i] = n - overlap(string, perms[i])
        where overlap is the maximum overlap length.

        Returns:
            Cost array of shape (m,)
        """
        costs = np.zeros(self.m, dtype=np.float32)
        for i, word in enumerate(self.perms_as_lists):
            if len(self.string) == 0:
                overlap = 0
            else:
                overlap = max_overlap(self.string, word)
            costs[i] = float(self.n - overlap)
        return costs

    def _get_observation(self) -> Dict[str, Any]:
        """Build the current observation."""
        # Coverage: boolean array
        coverage = self.coverage.copy().astype(np.int8)

        # Costs: already computed in self.costs
        costs = self.costs.copy()

        # Length: normalized to [0, 1]
        length_norm = np.array([min(1.0, len(self.string) / self.max_length)], dtype=np.float32)

        return {
            "coverage": coverage,
            "costs": costs,
            "length": length_norm,
        }

    def reset(
        self,
        seed: Optional[int] = None,
        options: Optional[dict] = None,
    ) -> tuple[Dict[str, Any], Dict[str, Any]]:
        """Reset the environment."""
        super().reset(seed=seed)

        self.string = []
        self.coverage[:] = False
        self.step_count = 0
        self.costs = self._compute_cost_vector()

        obs = self._get_observation()
        info = {"n": self.n}

        return obs, info

    def step(self, action: int) -> tuple[Dict[str, Any], float, bool, bool, Dict[str, Any]]:
        """
        Execute one step.

        Args:
            action: Index of permutation to add (0 to m-1)

        Returns:
            observation, reward, terminated, truncated, info
        """
        # Validate action
        if not self.action_space.contains(action):
            raise ValueError(f"Invalid action: {action}")

        i = int(action)
        word = self.perms_as_lists[i]

        # Compute overlap and cost
        if len(self.string) == 0:
            overlap = 0
        else:
            overlap = max_overlap(self.string, word)
        cost = self.n - overlap

        # Merge word into string
        self.string = merge_with_overlap(self.string, word)
        self.step_count += 1

        # Update coverage
        delta_new_perms = update_coverage_for_string(
            string=self.string,
            n=self.n,
            perms=self.perms,
            coverage=self.coverage,
            search_window=2 * self.n,  # Only check recent suffix
        )

        # Compute reward
        step_length_penalty = -float(cost)*0.2
        new_perm_reward = self.alpha_new_perm_reward * float(delta_new_perms)
        reward = step_length_penalty + new_perm_reward

        # Check termination conditions
        success = False
        truncated = False
        terminated = False

        if self.coverage.sum() == self.m:
            # All permutations covered
            success = True
            terminated = True
            final_length = len(self.string)
            reward += self.goal_bonus - self.length_penalty_at_goal * final_length
        elif len(self.string) > self.max_length or self.step_count >= self.max_steps:
            truncated = True
            terminated = True
            reward-=100
        else:
            terminated = False

        # Recompute cost vector for next step
        self.costs = self._compute_cost_vector()

        # Build observation
        obs = self._get_observation()

        # Build info
        info = {
            "n": self.n,
            "step_count": self.step_count,
            "current_length": len(self.string),
            "coverage_ratio": float(self.coverage.sum()) / float(self.m),
            "success": success,
            "truncated": truncated,
        }

        if terminated or truncated:
            info["final_length"] = len(self.string)
            info["sequence"] = list(self.string)
            info["covered_permutations"] = int(self.coverage.sum())

        return obs, reward, terminated, truncated, info




In [42]:
# Source: /content/utils/types.py
"""Type definitions for superpermutation RL research."""

from typing import Literal

# Environment type
EnvType = Literal["symbol", "word_cost"]

# Algorithm name
AlgoName = Literal["ppo", "dqn","random"]



In [43]:
# Source: /content/utils/common.py
"""Common utility functions for superpermutation RL research."""

import os
import json
import numpy as np
import torch
from typing import Dict, Any, Optional
import gymnasium as gym




def convert_numpy_types(obj: Any) -> Any:
    """
    Convert numpy types to Python native types for JSON serialization.

    Args:
        obj: Object that may contain numpy types

    Returns:
        Object with numpy types converted to Python types
    """
    if isinstance(obj, (np.integer, np.intc, np.intp, np.int8,
                        np.int16, np.int32, np.int64, np.uint8, np.uint16,
                        np.uint32, np.uint64)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float16, np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_numpy_types(item) for item in obj]
    else:
        return obj


def generate_run_name(
    env_type: EnvType,
    n: int,
    algo_name: AlgoName,
    seed: int,
    hyperparams: Optional[Dict[str, Any]] = None,
) -> str:
    """
    Generate a run name from parameters.

    Args:
        env_type: Environment type
        n: Permutation size
        algo_name: Algorithm name
        seed: Random seed
        hyperparams: Optional hyperparameters dict

    Returns:
        Run name string
    """
    run_name = f"{env_type}_n{n}_{algo_name}_seed{seed}"
    if hyperparams:
        # Add hyperparam signature to run name
        hp_sig = "_".join(f"{k}{v}" for k, v in sorted(hyperparams.items()))
        run_name = f"{run_name}_{hp_sig}"
    return run_name


def create_metadata(
    env_type: EnvType,
    n: int,
    algo_name: AlgoName,
    hyperparams: Dict[str, Any],
    seed: int,
    **kwargs: Any,
) -> Dict[str, Any]:
    """
    Create metadata dictionary for a run.

    Args:
        env_type: Environment type
        n: Permutation size
        algo_name: Algorithm name
        hyperparams: Hyperparameters dict
        seed: Random seed
        **kwargs: Additional metadata fields

    Returns:
        Metadata dictionary
    """
    metadata = {
        "env_type": env_type,
        "n": n,
        "algo_name": algo_name,
        "hyperparams": hyperparams,
        "seed": seed,
    }
    metadata.update(kwargs)
    return metadata


def save_metadata(run_dir: str, metadata: Dict[str, Any]) -> None:
    """
    Save metadata to a JSON file.

    Args:
        run_dir: Run directory path
        metadata: Metadata dictionary
    """
    # Convert numpy types to Python native types for JSON serialization
    metadata_serializable = convert_numpy_types(metadata)

    metadata_path = os.path.join(run_dir, "metadata.json")
    with open(metadata_path, "w") as f:
        json.dump(metadata_serializable, f, indent=2)


def detect_env_type(env: gym.Env) -> EnvType:
    """
    Detect environment type from an environment instance.

    Args:
        env: Gymnasium environment

    Returns:
        Environment type ("symbol" or "word_cost")
    """
    if hasattr(env, "costs"):
        return "word_cost"
    elif hasattr(env, "n") and hasattr(env, "perms"):
        return "symbol"
    else:
        # Fallback: check class name
        class_name = env.__class__.__name__
        if "Word" in class_name or "word" in class_name.lower():
            return "word_cost"
        else:
            return "symbol"


def set_all_seeds(seed: int) -> None:
    """
    Set all random seeds for reproducibility.

    Args:
        seed: Random seed value
    """
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)



In [44]:
# Source: /content/train_one.py
"""Train a single RL run."""

import os
import time
import numpy as np
import torch
from stable_baselines3 import PPO, A2C, DQN
from stable_baselines3.common.logger import configure
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from gymnasium import spaces




ALGO_MAP = {
    "ppo": PPO,
    "dqn": DQN,  # basic: dueling=False
}


def word_env_mask_fn(obs) -> np.ndarray:
    """
    Action mask function for WordCostSuperpermEnv.

    Masks out actions where:
    - The permutation is already covered AND
    - The cost equals the maximum cost (n, meaning zero overlap)

    Args:
        obs: Observation dict from WordCostSuperpermEnv

    Returns:
        Boolean mask of shape (m,) where False means invalid action
    """
    coverage = obs["coverage"]
    costs = obs["costs"]

    # Maximum cost should be n (alphabet size), which occurs when overlap is 0
    # We can infer n from the maximum cost value
    max_cost = np.max(costs)

    # Mask: allow all actions by default
    mask = np.ones(len(coverage), dtype=bool)

    # Mask out covered permutations with maximum cost (zero overlap, already covered)
    # This prevents adding a permutation that's already covered with no benefit
    mask = ~((coverage == 1) & (np.abs(costs - max_cost) < 0.01))

    return mask


def make_env(env_type: EnvType, n: int, use_mask: bool = False):
    """
    Create an environment.

    Args:
        env_type: "symbol" or "word_cost"
        n: Alphabet size
        use_mask: Whether to apply action masking (for MaskablePPO on WordEnv)

    Returns:
        Gymnasium environment
    """
    if env_type == "symbol":
        env = SymbolSuperpermEnv(n=n)
        return env
    elif env_type == "word_cost":
        env = WordCostSuperpermEnv(n=n)
        if use_mask:
            env = ActionMasker(env, word_env_mask_fn)
        return env
    else:
        raise ValueError(f"Unknown env_type: {env_type}")


def train_one_run(
    env_type: EnvType,
    n: int,
    algo_name: AlgoName,
    hyperparams: dict,
    seed: int,
    total_timesteps: int,
    log_root: str = "logs",
) -> str:
    """
    Train a single RL run.

    Args:
        env_type: "symbol" or "word_cost"
        n: Alphabet size
        algo_name: Algorithm name ("ppo", "maskable_ppo", "a2c", "dqn", "dqn_dueling")
        hyperparams: Hyperparameters dict
        seed: Random seed
        total_timesteps: Total training timesteps
        log_root: Root directory for logs
    """
    # Create run directory
    run_name = generate_run_name(env_type, n, algo_name, seed, hyperparams)
    run_dir = os.path.join(log_root, run_name)
    os.makedirs(run_dir, exist_ok=True)

    # Create environment
    use_mask = (algo_name == "maskable_ppo" and env_type == "word_cost")
    env = make_env(env_type, n, use_mask=use_mask)

    # Set seeds
    env.reset(seed=seed)
    set_all_seeds(seed)

    # Get algorithm class
    AlgoClass = ALGO_MAP.get(algo_name)
    if AlgoClass is None:
        raise ValueError(f"Unknown algorithm: {algo_name}")

    # Determine device (GPU if available, else CPU)
    cuda_available = torch.cuda.is_available()
    if cuda_available:
        device = "cuda"
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = "cpu"
        print("CUDA not available, using CPU")
        print("Note: To use GPU, ensure PyTorch with CUDA support is installed")

    # Check observation space type (handle wrapped environments)
    obs_space = env.observation_space
    # If wrapped, get the actual observation space
    while hasattr(obs_space, "observation_space"):
        obs_space = obs_space.observation_space

    # Determine policy type
    # Check if observation space is a Dict (not a Python dict, but gymnasium.spaces.Dict)
    if isinstance(obs_space, spaces.Dict):
        policy = "MultiInputPolicy"
    else:
        policy = "MlpPolicy"

    # Configure policy kwargs
    policy_kwargs = {}
    if algo_name == "dqn":
        policy_kwargs = {"policy_kwargs": {"dueling": False}}
    elif algo_name == "dqn_dueling":
        policy_kwargs = {"policy_kwargs": {"dueling": True}}

    # Configure SB3 logger
    logger = configure(run_dir, ["csv", "tensorboard"])

    # Create model
    model = AlgoClass(
        policy,
        env,
        verbose=1,
        seed=seed,
        device=device,
        tensorboard_log=os.path.join(run_dir, "tensorboard"),
        **hyperparams,
        **policy_kwargs,
    )
    model.set_logger(logger)

    # Create callback (save episodes for successful runs)
    episodes_dir = os.path.join(run_dir, "episodes")
    callback = SuperpermEpisodeCallback(verbose=1, save_episodes_dir=episodes_dir)

    # Train
    start_time = time.time()
    model.learn(
        total_timesteps=total_timesteps,
        callback=callback,
        log_interval=10,
    )
    wall_time = time.time() - start_time

    # Save model
    model.save(os.path.join(run_dir, "model"))

    # Save metadata
    metadata = create_metadata(
        env_type=env_type,
        n=n,
        algo_name=algo_name,
        hyperparams=hyperparams,
        seed=seed,
        total_timesteps=total_timesteps,
        wall_time=wall_time,
    )
    save_metadata(run_dir, metadata)

    return run_dir



In [45]:
# Source: /content/metrics.py
"""Aggregate metrics and extract top-3 superpermutations."""

import os
import json
import csv
import ast
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List, Any, Tuple
from collections import defaultdict




def scan_logs(log_root: str = "logs") -> List[Dict[str, Any]]:
    """
    Scan logs directory for all run directories.

    Returns:
        List of run metadata dicts
    """
    runs = []
    log_path = Path(log_root)

    if not log_path.exists():
        print(f"Log directory {log_root} does not exist.")
        return runs

    for run_dir in log_path.iterdir():
        if not run_dir.is_dir():
            continue

        metadata_path = run_dir / "metadata.json"
        progress_path = run_dir / "progress.csv"

        if not progress_path.exists():
            continue

        metadata = {}
        if metadata_path.exists():
            with open(metadata_path, "r") as f:
                metadata = json.load(f)

        runs.append({
            "run_dir": str(run_dir),
            "run_name": run_dir.name,
            "metadata": metadata,
            "progress_path": str(progress_path),
        })

    return runs


def load_progress_csv(csv_path: str) -> pd.DataFrame:
    """Load progress CSV file."""
    try:
        df = pd.read_csv(csv_path)
        return df
    except Exception as e:
        print(f"Error loading {csv_path}: {e}")
        return pd.DataFrame()


def extract_sequences_from_progress(df: pd.DataFrame, run_dir: str) -> List[Dict[str, Any]]:
    """
    Extract sequences from progress CSV.

    Note: progress.csv may not have sequences directly.
    We'll need to check if sequences are stored elsewhere or reconstruct from episodes.
    For now, we'll return empty list and note that sequences should be extracted
    from episode info during training.
    """
    sequences = []

    # If progress.csv has sequence column, use it
    if "sequence" in df.columns:
        for idx, row in df.iterrows():
            if row.get("success", 0) == 1:
                try:
                    seq_str = row["sequence"]
                    if isinstance(seq_str, str):
                        # Try to parse as Python literal (list)
                        seq = ast.literal_eval(seq_str)
                    else:
                        seq = seq_str
                    sequences.append({
                        "episode": int(row.get("episode", idx)),
                        "final_length": int(row.get("final_length", 0)),
                        "episode_return": float(row.get("episode_return", 0.0)),
                        "sequence": seq,
                    })
                except (ValueError, SyntaxError, TypeError):
                    # Skip if parsing fails
                    pass

    return sequences


def extract_sequences_from_episodes(run_dir: str, n: int) -> List[Dict[str, Any]]:
    """
    Extract sequences from episode logs.

    Looks for episode JSON files in the episodes/ subdirectory.
    """
    sequences = []
    run_path = Path(run_dir)

    # Check for episode files in episodes/ subdirectory
    episodes_dir = run_path / "episodes"
    if episodes_dir.exists():
        episode_files = list(episodes_dir.glob("episode_*.json"))
        for ep_file in sorted(episode_files):
            try:
                with open(ep_file, "r") as f:
                    ep_data = json.load(f)
                    if ep_data.get("success", False) and "sequence" in ep_data:
                        sequences.append({
                            "episode": ep_data.get("episode", 0),
                            "final_length": ep_data.get("final_length", 0),
                            "episode_return": ep_data.get("episode_return", 0.0),
                            "sequence": ep_data["sequence"],
                        })
            except Exception as e:
                print(f"Error reading {ep_file}: {e}")
                pass

    return sequences


def get_top3_superperms(
    sequences: List[Dict[str, Any]],
    n: int,
) -> List[Dict[str, Any]]:
    """
    Get top-3 shortest distinct superpermutations.

    Args:
        sequences: List of sequence dicts with keys: episode, final_length, episode_return, sequence
        n: Alphabet size

    Returns:
        List of top-3 superpermutation dicts
    """
    if not sequences:
        return []

    # Canonicalize and deduplicate
    seen = set()
    unique_seqs = []

    for seq_data in sequences:
        seq = seq_data["sequence"]
        if not isinstance(seq, list):
            continue

        canonical = tuple(canonicalize_superperm(seq, n))
        if canonical not in seen:
            seen.add(canonical)
            unique_seqs.append({
                "canonical_sequence": list(canonical),
                "original_sequence": seq,
                "final_length": seq_data["final_length"],
                "episode_return": seq_data.get("episode_return", 0.0),
                "episode": seq_data.get("episode", 0),
            })

    # Sort by final_length, then by episode_return (descending)
    unique_seqs.sort(key=lambda x: (x["final_length"], -x["episode_return"]))

    # Take top 3
    top3 = unique_seqs[:3]

    # Add rank
    for i, item in enumerate(top3, 1):
        item["rank"] = i

    return top3


def compute_run_metrics(df: pd.DataFrame) -> Dict[str, Any]:
    """Compute aggregated metrics for a run."""
    if df.empty:
        return {}

    metrics = {}

    # Final length statistics
    if "final_length" in df.columns:
        metrics["mean_final_length"] = float(df["final_length"].mean())
        metrics["min_final_length"] = float(df["final_length"].min())
        metrics["max_final_length"] = float(df["final_length"].max())
        metrics["std_final_length"] = float(df["final_length"].std())

    # Success rate
    if "success" in df.columns:
        success_count = int(df["success"].sum())
        total_episodes = len(df)
        metrics["success_rate"] = float(success_count / total_episodes) if total_episodes > 0 else 0.0
        metrics["success_count"] = success_count
        metrics["total_episodes"] = total_episodes

    # Coverage ratio
    if "coverage_ratio" in df.columns:
        metrics["mean_coverage_ratio"] = float(df["coverage_ratio"].mean())

    # Episode steps
    if "episode_steps" in df.columns:
        metrics["mean_episode_steps"] = float(df["episode_steps"].mean())

    # Episode return
    if "episode_return" in df.columns:
        metrics["mean_episode_return"] = float(df["episode_return"].mean())
        metrics["std_episode_return"] = float(df["episode_return"].std())
        metrics["min_episode_return"] = float(df["episode_return"].min())
        metrics["max_episode_return"] = float(df["episode_return"].max())

    # Steps to first success
    if "success" in df.columns and "episode_steps" in df.columns:
        success_episodes = df[df["success"] == 1]
        if not success_episodes.empty:
            first_success_idx = success_episodes.index[0]
            metrics["episodes_to_first_success"] = int(first_success_idx)
            # Cumulative steps to first success
            metrics["steps_to_first_success"] = int(df.loc[:first_success_idx, "episode_steps"].sum())
        else:
            metrics["episodes_to_first_success"] = None
            metrics["steps_to_first_success"] = None

    return metrics


def update_metrics_for_run(
    run_dir: str,
    log_root: str = "logs",
    output_dir: str = "metrics",
) -> None:
    """
    Update metrics for a single run (incremental update).

    This function updates the aggregated metrics after a single run completes.
    It reads existing metrics and updates them with the new run's data.

    Args:
        run_dir: Directory of the completed run
        log_root: Root directory containing run logs
        output_dir: Directory to save aggregated metrics
    """
    os.makedirs(output_dir, exist_ok=True)

    # Load metadata and progress for this run
    run_path = Path(run_dir)
    metadata_path = run_path / "metadata.json"
    progress_path = run_path / "progress.csv"

    if not progress_path.exists():
        return

    metadata = {}
    if metadata_path.exists():
        with open(metadata_path, "r") as f:
            metadata = json.load(f)

    env_type = metadata.get("env_type", "unknown")
    n = metadata.get("n", 0)
    algo_name = metadata.get("algo_name", "unknown")
    hyperparams = metadata.get("hyperparams", {})

    # Create group key
    hp_sig = "_".join(f"{k}{v}" for k, v in sorted(hyperparams.items())) if hyperparams else "default"
    group_key = (env_type, n, algo_name, hp_sig)
    group_name = f"{env_type}_n{n}_{algo_name}_{hp_sig}"

    # Load existing summary if exists
    summary_path = os.path.join(output_dir, "metrics_summary.json")
    existing_summaries = []
    if os.path.exists(summary_path):
        with open(summary_path, "r") as f:
            existing_summaries = json.load(f)

    # Find existing summary for this group
    existing_summary = None
    existing_idx = None
    for idx, summary in enumerate(existing_summaries):
        if (summary.get("env_type") == env_type and
            summary.get("n") == n and
            summary.get("algo_name") == algo_name and
            summary.get("hyperparams") == hp_sig):
            existing_summary = summary
            existing_idx = idx
            break

    # Get all runs in this group
    all_runs = scan_logs(log_root)
    group_runs = []
    for run in all_runs:
        run_meta = run["metadata"]
        run_env_type = run_meta.get("env_type", "unknown")
        run_n = run_meta.get("n", 0)
        run_algo = run_meta.get("algo_name", "unknown")
        run_hp = run_meta.get("hyperparams", {})
        run_hp_sig = "_".join(f"{k}{v}" for k, v in sorted(run_hp.items())) if run_hp else "default"

        if (run_env_type == env_type and run_n == n and
            run_algo == algo_name and run_hp_sig == hp_sig):
            group_runs.append(run)

    # Aggregate metrics for this group
    all_metrics = []
    all_sequences = []

    for run in group_runs:
        df = load_progress_csv(run["progress_path"])
        if df.empty:
            continue

        # Compute run-level metrics
        run_metrics = compute_run_metrics(df)
        run_metrics["run_name"] = run["run_name"]
        run_metrics["seed"] = run["metadata"].get("seed", None)
        run_metrics["wall_time"] = run["metadata"].get("wall_time", None)
        all_metrics.append(run_metrics)

        # Extract sequences
        sequences = extract_sequences_from_progress(df, run["run_dir"])
        if not sequences:
            sequences = extract_sequences_from_episodes(run["run_dir"], n)

        for seq in sequences:
            seq["run_name"] = run["run_name"]
            all_sequences.append(seq)

    if not all_metrics:
        return

    # Aggregate across runs
    metrics_df = pd.DataFrame(all_metrics)

    summary = {
        "env_type": env_type,
        "n": n,
        "algo_name": algo_name,
        "hyperparams": hp_sig,
        "num_runs": len(group_runs),
        "mean_final_length": float(metrics_df["mean_final_length"].mean()) if "mean_final_length" in metrics_df.columns else None,
        "min_final_length": float(metrics_df["min_final_length"].min()) if "min_final_length" in metrics_df.columns else None,
        "max_final_length": float(metrics_df["max_final_length"].max()) if "max_final_length" in metrics_df.columns else None,
        "mean_success_rate": float(metrics_df["success_rate"].mean()) if "success_rate" in metrics_df.columns else None,
        "mean_coverage_ratio": float(metrics_df["mean_coverage_ratio"].mean()) if "mean_coverage_ratio" in metrics_df.columns else None,
        "mean_episode_steps": float(metrics_df["mean_episode_steps"].mean()) if "mean_episode_steps" in metrics_df.columns else None,
        "mean_episode_return": float(metrics_df["mean_episode_return"].mean()) if "mean_episode_return" in metrics_df.columns else None,
        "total_wall_time": float(metrics_df["wall_time"].sum()) if "wall_time" in metrics_df.columns else None,
    }

    # Update or add summary
    if existing_idx is not None:
        existing_summaries[existing_idx] = summary
    else:
        existing_summaries.append(summary)

    # Save updated summary
    with open(summary_path, "w") as f:
        json.dump(existing_summaries, f, indent=2)

    # Save CSV version
    summary_df = pd.DataFrame(existing_summaries)
    csv_path = os.path.join(output_dir, "metrics_summary.csv")
    summary_df.to_csv(csv_path, index=False)

    # Extract and save top-3 superpermutations for this group
    top3 = get_top3_superperms(all_sequences, n)
    if top3:
        top3_path = os.path.join(output_dir, f"{group_name}_top3_superperms.json")
        with open(top3_path, "w") as f:
            json.dump(top3, f, indent=2)
        print(f"Updated metrics and top-3 for {group_name} ({len(group_runs)} runs)")


def aggregate_metrics(log_root: str = "logs", output_dir: str = "metrics") -> None:
    """
    Aggregate metrics from all runs and extract top-3 superpermutations.

    Args:
        log_root: Root directory containing run logs
        output_dir: Directory to save aggregated metrics
    """
    os.makedirs(output_dir, exist_ok=True)

    runs = scan_logs(log_root)
    print(f"Found {len(runs)} runs")

    # Group runs by (env_type, n, algo_name, hyperparams)
    grouped_runs = defaultdict(list)

    for run in runs:
        metadata = run["metadata"]
        env_type = metadata.get("env_type", "unknown")
        n = metadata.get("n", 0)
        algo_name = metadata.get("algo_name", "unknown")
        hyperparams = metadata.get("hyperparams", {})

        # Create group key
        hp_sig = "_".join(f"{k}{v}" for k, v in sorted(hyperparams.items())) if hyperparams else "default"
        group_key = (env_type, n, algo_name, hp_sig)

        grouped_runs[group_key].append(run)

    # Aggregate metrics for each group
    summary_rows = []

    for group_key, group_runs in grouped_runs.items():
        env_type, n, algo_name, hp_sig = group_key

        # Aggregate metrics across all runs in group
        all_metrics = []
        all_sequences = []

        for run in group_runs:
            df = load_progress_csv(run["progress_path"])
            if df.empty:
                continue

            # Compute run-level metrics
            run_metrics = compute_run_metrics(df)
            run_metrics["run_name"] = run["run_name"]
            run_metrics["seed"] = run["metadata"].get("seed", None)
            run_metrics["wall_time"] = run["metadata"].get("wall_time", None)
            all_metrics.append(run_metrics)

            # Extract sequences (try multiple methods)
            sequences = extract_sequences_from_progress(df, run["run_dir"])
            if not sequences:
                sequences = extract_sequences_from_episodes(run["run_dir"], n)

            for seq in sequences:
                seq["run_name"] = run["run_name"]
                all_sequences.append(seq)

        if not all_metrics:
            continue

        # Aggregate across runs
        metrics_df = pd.DataFrame(all_metrics)

        summary = {
            "env_type": env_type,
            "n": n,
            "algo_name": algo_name,
            "hyperparams": hp_sig,
            "num_runs": len(group_runs),
            "mean_final_length": float(metrics_df["mean_final_length"].mean()) if "mean_final_length" in metrics_df.columns else None,
            "min_final_length": float(metrics_df["min_final_length"].min()) if "min_final_length" in metrics_df.columns else None,
            "max_final_length": float(metrics_df["max_final_length"].max()) if "max_final_length" in metrics_df.columns else None,
            "mean_success_rate": float(metrics_df["success_rate"].mean()) if "success_rate" in metrics_df.columns else None,
            "mean_coverage_ratio": float(metrics_df["mean_coverage_ratio"].mean()) if "mean_coverage_ratio" in metrics_df.columns else None,
            "mean_episode_steps": float(metrics_df["mean_episode_steps"].mean()) if "mean_episode_steps" in metrics_df.columns else None,
            "mean_episode_return": float(metrics_df["mean_episode_return"].mean()) if "mean_episode_return" in metrics_df.columns else None,
            "total_wall_time": float(metrics_df["wall_time"].sum()) if "wall_time" in metrics_df.columns else None,
        }

        summary_rows.append(summary)

        # Extract top-3 superpermutations for this group
        top3 = get_top3_superperms(all_sequences, n)

        if top3:
            # Save top-3 for this group
            group_name = f"{env_type}_n{n}_{algo_name}_{hp_sig}"
            top3_path = os.path.join(output_dir, f"{group_name}_top3_superperms.json")
            with open(top3_path, "w") as f:
                json.dump(top3, f, indent=2)
            print(f"Saved top-3 superpermutations for {group_name}")

    # Save summary CSV
    summary_df = pd.DataFrame(summary_rows)
    summary_path = os.path.join(output_dir, "metrics_summary.csv")
    summary_df.to_csv(summary_path, index=False)
    print(f"Saved metrics summary to {summary_path}")

    # Also save as JSON
    summary_json_path = os.path.join(output_dir, "metrics_summary.json")
    with open(summary_json_path, "w") as f:
        json.dump(summary_rows, f, indent=2)
    print(f"Saved metrics summary to {summary_json_path}")


if __name__ == "__main__":
    aggregate_metrics()



Log directory logs does not exist.
Found 0 runs
Saved metrics summary to metrics/metrics_summary.csv
Saved metrics summary to metrics/metrics_summary.json


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [46]:
# Source: /content/sweep.py
"""Run full experiment sweep over all combinations."""

import os
import itertools



def generate_hyperparam_combinations(hyper_grid: dict) -> list[dict]:
    """Generate all combinations of hyperparameters."""
    if not hyper_grid:
        return [{}]

    keys = list(hyper_grid.keys())
    values = list(hyper_grid.values())
    combinations = []
    for combo in itertools.product(*values):
        combinations.append(dict(zip(keys, combo)))
    return combinations


def run_sweep(log_root: str = "logs") -> None:
    """Run the full experiment sweep."""
    total_runs = 0

    for n in N_LIST:
        for env_type in ENV_TYPES:
            for algo_name in ALGOS:
                # Check if should skip this combination
                if should_skip(algo_name, env_type, n):
                    print(f"Skipping {algo_name} for {env_type} n={n} (skip condition)")
                    continue

                if algo_name in ["random", "greedy"]:
                    # Non-learning baselines
                    for seed in SEEDS:
                        run_name = generate_run_name(env_type, n, algo_name, seed, None)
                        run_dir = os.path.join(log_root, run_name)
                        os.makedirs(run_dir, exist_ok=True)

                        # Create environment using common function
                        env = make_env(env_type, n, use_mask=False)

                        # Run baseline
                        if algo_name == "random":
                            run_random(
                                env=env,
                                total_episodes=BASELINE_EPISODES["random"],
                                max_steps_per_episode=env.max_steps,
                                log_dir=run_dir,
                                seed=seed,
                            )
                        elif algo_name == "greedy":
                            run_greedy(
                                env=env,
                                total_episodes=BASELINE_EPISODES["greedy"],
                                max_steps_per_episode=env.max_steps,
                                log_dir=run_dir,
                                seed=seed,
                            )

                        total_runs += 1
                        print(f"Completed: {run_name} ({total_runs} total runs)")

                else:
                    # RL algorithms
                    # Get hyperparameter combinations
                    hyper_grid = HYPER_GRIDS.get(algo_name, {})
                    hyper_combos = generate_hyperparam_combinations(hyper_grid)

                    # Get timesteps for this algorithm
                    total_timesteps = TIMESTEPS.get(algo_name)
                    if total_timesteps is None:
                        print(f"Warning: No timesteps defined for {algo_name}, skipping")
                        continue

                    for hyperparams in hyper_combos:
                        for seed in SEEDS:
                            try:
                                run_dir = train_one_run(
                                    env_type=env_type,
                                    n=n,
                                    algo_name=algo_name,
                                    hyperparams=hyperparams,
                                    seed=seed,
                                    total_timesteps=total_timesteps,
                                    log_root=log_root,
                                )
                                total_runs += 1
                                print(f"Completed: {env_type}_n{n}_{algo_name}_seed{seed} ({total_runs} total runs)")
                            except Exception as e:
                                print(f"Error in {env_type}_n{n}_{algo_name}_seed{seed}: {e}")
                                continue

    print(f"\nSweep complete! Total runs: {total_runs}")

    # Aggregate metrics after all runs are complete
    print("\nAggregating metrics...")
    try:
        aggregate_metrics(log_root=log_root)
        print("Metrics aggregation complete!")
    except Exception as e:
        print(f"Error aggregating metrics: {e}")


if __name__ == "__main__":
    run_sweep()




Using GPU: NVIDIA L4
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Completed: word_cost_n5_ppo_seed0 (1 total runs)
Using GPU: NVIDIA L4
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Completed: symbol_n5_ppo_seed0 (2 total runs)

Sweep complete! Total runs: 2

Aggregating metrics...
Found 2 runs
Saved top-3 superpermutations for symbol_n5_ppo_gamma0.99_learning_rate0.0002
Saved metrics summary to metrics/metrics_summary.csv
Saved metrics summary to metrics/metrics_summary.json
Metrics aggregation complete!


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [47]:
!zip -r logs.zip logs

  adding: logs/ (stored 0%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/ (stored 0%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/model.zip (stored 0%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/episodes/ (stored 0%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/episodes/episode_7162.json (deflated 82%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/episodes/episode_6863.json (deflated 81%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/episodes/episode_7085.json (deflated 82%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/episodes/episode_7183.json (deflated 82%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/episodes/episode_6723.json (deflated 82%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_rate0.0002/events.out.tfevents.1765094783.fe29f4c9841d.1256.4 (deflated 76%)
  adding: logs/symbol_n5_ppo_seed0_gamma0.99_learning_